# Import Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load the dataset using pandas
data = pd.read_csv("VOW3.DE_max.csv")

In [3]:
data = data.tail(120)

# Polynomial Regression Analysis

## Imports

In [4]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model
import time
import datetime

## Loading Data into np array

In [5]:
stocks_X = []
stocks_Y = []

for index, row in data.iterrows():
    element = datetime.datetime.strptime(row['Date'],"%Y-%m-%d")
    timestamp = datetime.datetime.timestamp(element)
    stocks_X.append([timestamp]) 
    stocks_Y.append([row['Close']])

stocks_X = np.array(stocks_X)
stocks_Y = np.array(stocks_Y)

stocks_X_train = stocks_X[:-30]
stocks_X_test = stocks_X[-30:]

stocks_Y_train = stocks_Y[:-30]
stocks_Y_test = stocks_Y[-30:]

## Normalizing

In [6]:
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#rescaled_data = scaler.fit_transform(stocks_X)

#print('After Rescaling:')
#np.set_printoptions(precision=3)
#rescaled_data

## Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

#m = 90
#X = 6 * np.random.rand(m, 1) - 3
X = stocks_X_train
Y = stocks_Y_train

#print(X)
#y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)
#print(Y)

poly_features = PolynomialFeatures(degree=2, include_bias=False)

X_poly = poly_features.fit_transform(X)
print(X_poly.shape)
print(y.shape)

X_poly[0]

lin_reg = linear_model.LinearRegression()

lin_reg.fit(X_poly, Y)

lin_reg.intercept_, lin_reg.coef_

## Plot

In [ ]:
%matplotlib inline


# Plot outputs
plt.scatter(stocks_X, stocks_Y, 10, color='black', alpha=1)
#plt.plot(stocks_X_test, stocks_Y_pred, color='red', linewidth=2.5)
#plt.plot(stocks_X_train, stocks_Y_pred_train, color='green', linewidth=2.5)
#plt.plot(X_poly[:,0],regr.predict(X_poly),'-r')
a = lin_reg.coef_[0][1]
b = lin_reg.coef_[0][0]
c = lin_reg.intercept_[0]
print(a)
print(b)
print(c)
x = np.linspace(min(stocks_X), max(stocks_X), 256, endpoint = True)
#print(x)
y = (a * (x * x)) + (b * x) + c
plt.plot(x, y, '-g', label=r'$y = 3x^2 + 4x + 2$')

    
plt.xlim(min(stocks_X),max(stocks_X))
plt.ylim(min(stocks_Y),max(stocks_Y))
plt.ylabel("Preis")
plt.xlabel("Zeit")

plt.xticks(())
plt.yticks(())

plt.show()